In [ ]:
using Pkg
Pkg.activate("/home/bertschi/GitRepos/FinNetValu/")

In [ ]:
using FinNetValu
using Distributions
using LinearAlgebra
using SparseArrays
using ForwardDiff
using Plots
using DataFrames
using DataFramesMeta
using CSV

# pyplot()

In [ ]:
r = 0.0
τ = 1.0
σ = [0.2, 0.2]
Lᵨ = cholesky([1.0 0; 0 1.0]).L

θ = BlackScholesParams(r, τ, σ, Lᵨ)

In [ ]:
function debtval(net, a₀, θ)
    N = numfirms(net)
    expectation(Z -> discount(θ) .* debtview(net, fixvalue(net, Aτ(a₀, θ, Z); m = 0)),
                MonteCarloSampler(MvNormal(N, 1.0)),
                25000)
end

function insurancecost(net, a₀, θ)
    1 .- debtval(net, a₀, θ) ./ (discount(θ) .* net.d)
end

In [ ]:
function create_net(di₁, de₁, di₂, de₂)
    d = [di₁ + de₁, di₂ + de₂]
    Mᵈ = [0 (di₂ / d[2]); (di₁ / d[1]) 0]
    XOSModel(spzeros(2,2), Mᵈ, I, d)
end

function create_net_sym(di, de)
    d = [di + de, di + de]
    Mᵈ = [0 (di / d[2]); (di / d[1]) 0]
    XOSModel(spzeros(2,2), Mᵈ, I, d)
end

In [ ]:
di₂ = 0.6
de₂ = 0.4
df = DataFrame(di₁ = Float64[], de₁ = Float64[], in1 = Float64[], in2 = Float64[], a0 = Float64[])

for di₁ in range(0.2, length = 11, stop = 1)
    for de₁ in range(0.2, length = 11, stop = 1)
        for a₀ in range(0.2, length = 5, stop = 0.8)
            net = create_net(di₁, de₁, di₂, de₂)
            ins = insurancecost(net, a₀, θ)
            push!(df, (di₁, de₁, ins[1], ins[2], a₀))
       end
    end
end

df

In [ ]:
CSV.write("/tmp/foo.csv", df)

In [ ]:
? XOSModel

In [ ]:
di₂ = 0.6
de₂ = 0.4
df = DataFrame(di1 = Float64[], de1 = Float64[], A1 = Float64[], A2 = Float64[], sol1 = Bool[], sol2 = Bool[])

for di₁ in range(0.01, length = 5, stop = 0.8)
    for de₁ in range(0.01, length = 5, stop = 0.8)
        net = create_net(di₁, de₁, di₂, de₂)
        # net = create_net_sym(di₁, de₁)
        # for A1 in range(0.0, length = 51, stop = 3)
        for logA1 in range(-2, length = 51, stop = 1)
            for logA2 in range(-2, length = 51, stop = 1)
                A = 10 .^ [logA1, logA2]
                x = fixvalue(net, A; m = 0)
                debt = debtview(net, x)
                sol  = solvent(net, x)
                ins = (net.d .- debt) ./ net.d
                push!(df, (di₁, de₁, A[1], A[2], sol[1], sol[2]))
            end
        end
    end
end

CSV.write("/tmp/baz.csv", df)

In [ ]:
di₂ = 0.6
de₂ = 0.4
df = DataFrame(di1 = Float64[], de1 = Float64[], in1 = Float64[], in2 = Float64[], lam1 = Float64[], lam2 = Float64[], sigma = Float64[], a0 = Float64[])

for di1 in range(0.01, length = 11, stop = 0.8)
    for de1 in range(0.01, length = 5, stop = 0.8)
        net = create_net(di1, de1, di₂, de₂)
        for sigma in [0.1, 0.2, 0.4]
            θ = BlackScholesParams(r, τ, sigma, Lᵨ)
            for a0 in [0.6, 0.8, 1, 1.2]
                N = numfirms(net)
                x = expectation(Z -> discount(θ) * fixvalue(net, Aτ(a0, θ, Z); m = 0),
                                MonteCarloSampler(MvNormal(N, 1.0)),
                                25000)
                v = equityview(net, x) .+ debtview(net, x)
                lam = equityview(net, x) ./ v
                ins = 1 .- debtview(net, x) ./ (discount(θ) .* net.d)
                push!(df, (di1, de1, ins[1], ins[2], lam[1], lam[2], sigma, a0))
            end
       end
    end
end

CSV.write("/tmp/qux.csv", df)

In [ ]:
using RCall

R"library(tidyverse)"
R"library(ggthemes)"
R"library(viridis)"

In [ ]:
R"read_csv(\"/tmp/baz.csv\") %>% ggplot(aes(A1, A2, fill = interaction(sol1, sol2))) + geom_tile() + scale_fill_colorblind() + theme_tufte() + facet_grid(de1 ~ di1) + scale_x_log10() + scale_y_log10() + geom_vline(xintercept = 1) + geom_hline(yintercept=1)"

In [ ]:
R"read_csv(\"/tmp/qux.csv\") %>% mutate(in12 = in1 + in2, lam12 = lam1 + lam2) %>% gather(key, val, matches(\"in\")) %>% ggplot(aes(di1, val, color = factor(sigma), linetype = factor(a0))) + geom_line() + facet_grid(de1 ~ key) + theme_tufte() + scale_color_colorblind()"

In [ ]:
@linq CSV.read("/tmp/qux.csv") |>
    transform(in12 = :in1 .+ :in2, lam12 = :lam1 .+ :lam2) |>
    stack([:in1, :in2, :in12]) |>
    with(plot(:di1, :value, group = (:sigma, :a0),
              layout = (5, 3)))

Three bank example 6.17 from the thesis of S. Karl showing non-monotonic effect of changes in nominal debt to single firm.

In [ ]:
ex6_17_net(d₃) = XOSModel([0 0 0.4; 0.2 0 0; 0 0 0], [0 0 0; 0.6 0 0.1; 0 0 0], LinearAlgebra.I, [6, 11.8, d₃])
ex6_17_a = [3, 7.2, 19]

In [ ]:
ex6_17_data = DataFrame(d₁ = 6, d₂ = 11.8, d₃ = 0.1:0.05:20)

function foo(d)
    net = ex6_17_net(d)
    debtview(net, fixvalue(net, ex6_17_a))
end
tmp = map(foo, ex6_17_data[:d₃])

@linq ex6_17_data |>
    transform(r₁ = map(x -> x[1], tmp),
              r₂ = map(x -> x[2], tmp),
              r₃ = map(x -> x[3], tmp)) |>
    transform(in₁ = 1 .- :r₁ ./ :d₁,
              in₂ = 1 .- :r₂ ./ :d₂,
              in₃ = 1 .- :r₃ ./ :d₃) |>
    transform(inall = :in₁ .+ :in₂ .+ :in₃) |>
    with(plot(:d₃, [:in₁, :in₂, :in₃, :inall]))

### Experiments with AutoDiff

In [ ]:
using ForwardDiff

In [ ]:
## Can we take derivatives of expectations?
f(a) = expectation(Z -> Aτ(a, θ, Z), MonteCarloSampler(Normal(0, 1)), 1000)
f([1.0, 2.0])

In [ ]:
reshape(ForwardDiff.jacobian(f, [1.0, 2.0]), 2, 2, 2)

In [ ]:
θ = BlackScholesParams(0., 1., 0.1)

In [ ]:
## Try on insurance costs ...
di = 0.05:0.05:1.0
din12 = [ForwardDiff.jacobian(di₁ -> insurancecost(create_net(di₁[1], 0.8, 0.6, 0.4), [0.6, 0.6], θ), [x])
         for x in di]
in12 = [insurancecost(create_net(x, 0.8, 0.6, 0.4), [0.6, 0.6], θ) for x in di]

In [ ]:
plt = plot()
plot!(plt, di, map(sum, din12))
plot!(plt, di, map(sum, in12))
plot!(plt, di, cumsum(map(sum, din12)) .* 0.05)

### Small benchmarks for NEVA derivatives

In [ ]:
function bench(N)
    Lᵉ = rand(Uniform(), N)
    L  = rand(Uniform(), N, N) .* (rand(Uniform(), N, N) .< 0.4)
    L[diagind(L)] .= 0
    net = EisenbergNoeModel(Lᵉ, L)
    
    a = rand(Uniform(), N)
    @time Ja = fixjacobian(net, a)
    @time Jb = ForwardDiff.jacobian(a -> fixvalue(net, a; m = 0, xtol = 1e-8, ftol = 1e-8), a)
    Ja, Jb
end

In [ ]:
Ja, Jb = bench(100)

In [ ]:
sum(Ja; dims = 1)

In [ ]:
sum(Jb; dims = 1)

In [ ]:
all(isapprox(Ja, Jb; rtol = 1e-6))

### Systemic insurance

Here we investigate an example where one bank increases its exposure with another. We evaluate the resulting per euro cost of insurance on all liabilities held outside the banking system.

In [ ]:
function create_net(di₁, de₁, di₂, de₂)
    d = [di₁ + de₁, di₂ + de₂]
    Mᵈ = [0 (di₂ / d[2]); (di₁ / d[1]) 0]
    XOSModel(spzeros(2,2), Mᵈ, I, d)
end

function create_net_sym(di, de)
    d = [di + de, di + de]
    Mᵈ = [0 (di / d[2]); (di / d[1]) 0]
    XOSModel(spzeros(2,2), Mᵈ, I, d)
end

function debtval(net, a₀, θ)
    N = numfirms(net)
    expectation(Z -> discount(θ) .* debtview(net, fixvalue(net, Aτ(a₀, θ, Z); m = 0)),
                MonteCarloSampler(MvNormal(N, 1.0)),
                25000)
end

function extinsurancecost(di₁, de₁, di₂, de₂, a₀, θ)
    net = create_net(di₁, de₁, di₂, de₂)
    1 .- (debtval(net, a₀, θ) .* [de₁, de₂] ./ [di₁ + de₁, di₂ + de₂]) ./ (discount(θ) .* [de₁, de₂])
end

function extinsurancecost_sym(di, de, a₀, θ)
    net = create_net_sym(di, de)
    1 .- (debtval(net, a₀, θ) .* [de, de] ./ [di + de, di + de]) ./ (discount(θ) .* [de, de])
end

In [ ]:
r = 0.0
τ = 1.0
Lᵨ = cholesky([1.0 0; 0 1.0]).L

di₂ = 0.6
de₂ = 0.4
df = DataFrame(di1 = Float64[], de1 = Float64[], in1 = Float64[], in2 = Float64[], sigma = Float64[], a0 = Float64[])

for di₁ in range(0.01, length = 11, stop = 0.8)
    for de₁ in range(0.01, length = 5, stop = 0.8)
        for σ in [0.1, 0.2, 0.4]
            θ = BlackScholesParams(r, τ, σ, Lᵨ)
            for a₀ in [0.4, 0.6, 0.8, 1.0]
                ins = extinsurancecost(di₁, de₁, di₂, de₂, a₀, θ)
                push!(df, (di₁, de₁, ins[1], ins[2], σ, a₀))
            end
       end
    end
end

CSV.write("/tmp/insu.csv", df)

In [ ]:
using RCall

R"library(tidyverse)"
R"library(ggthemes)"
R"library(viridis)"

In [ ]:
R"read_csv(\"/tmp/insu.csv\") %>%
    mutate(in12 = in1 + in2) %>%
    gather(key, val, matches(\"in\")) %>%
    ggplot(aes(di1, val,
               color = factor(sigma),
               linetype = factor(a0))) +
    geom_line() +
    facet_grid(de1 ~ key) +
    theme_tufte() +
    scale_color_colorblind()"

In [ ]:
df = DataFrame(di = Float64[], de = Float64[], in1 = Float64[], in2 = Float64[], sigma = Float64[], a0 = Float64[])

for di in range(0.01, length = 11, stop = 0.8)
    for de in range(0.01, length = 5, stop = 0.8)
        for σ in [0.1, 0.2, 0.4]
            θ = BlackScholesParams(r, τ, σ, Lᵨ)
            for a₀ in [0.4, 0.6, 0.8, 1.0]
                ins = extinsurancecost_sym(di, de, a₀, θ)
                push!(df, (di, de, ins[1], ins[2], σ, a₀))
            end
       end
    end
end

CSV.write("/tmp/insu_sym.csv", df)

In [ ]:
R"read_csv(\"/tmp/insu_sym.csv\") %>%
    mutate(in12 = in1 + in2) %>%
    gather(key, val, matches(\"in\")) %>%
    ggplot(aes(di, val,
               color = factor(sigma),
               linetype = factor(a0))) +
    geom_line() +
    facet_grid(de ~ key) +
    theme_tufte() +
    scale_color_colorblind()"